In [48]:
from bs4 import BeautifulSoup
import pandas as pd 
import requests
df = pd.DataFrame({"MLS": [],"Street": [], "City":[],"List Price":[],"Bedrooms":[],"Bathrooms":[],"Square Footage":[],"Date":[]})


In [49]:
def scraping(dataframe):
    page=requests.get("http://www.slocountyhomes.com/newlistex.php")
    data=BeautifulSoup(page.text,"html.parser")
    i=0
    date=""
    date="NA"
    MLS=""
    Street=""
    City=""
    List=""
    Bed=""
    Bath=""
    Sq=0
    Date=""
    for table in data.find_all('td'):
        if len(table.text.split("November"))!=1:
            date=table.text
        elif table.text.lstrip().rstrip()=="":
            i=i
        else:
            if i==0:
                MLS=table.text.lstrip().rstrip()
                i+=1
            elif i==1:
                Street=table.text.lstrip().rstrip()
                i+=1
            elif i==2:
                City=table.text.lstrip().rstrip()
                i+=1
            elif i==3:
                List=table.text.lstrip().rstrip()
                i+=1
            elif i==4:
                Bed=table.text.lstrip().rstrip()
                i+=1
            elif i==5:
                Bath=table.text.lstrip().rstrip()
                i+=1
            elif i==6:
                Sq=table.text.lstrip().rstrip()
                i=0
                dataframe.loc[len(dataframe)]=[Bath,Bed,City,date,List,MLS,Sq,Street]
                
    dataframe=dataframe.drop_duplicates()
    return dataframe

Lot size most important in prediction 


In [50]:
#for td in data.find_all('td'):
#    print(td.text.lstrip().rstrip())

In [51]:
df=scraping(df)

In [52]:
df

,Bathrooms,Bedrooms,City,Date,List Price,MLS,Square Footage,Street
0,2,2,Arroyo Grande,November 15,"$519,000",1072933,1746,306 Hondonada
1,2,3,Arroyo Grande,November 15,"$560,000",1072919,1608,327 Corona Del Terra
2,3,3,Atascadero,November 15,"$555,000",1072958,1902,7777 Del Rio
3,6,6,Cambria,November 15,"$1,390,000",1072528,3120,6050 Brighton
4,2,3,King City,November 15,"$215,000",1072953,1282,613 Maple
5,2,2,Lompoc,November 15,"$165,000",1072944,972,1213 Cypress
6,2,3,Los Osos,November 15,"$499,000",1072932,1080,1854 7th
7,2,4,Nipomo,November 15,"$549,900",1072938,1700,715 Crystal
8,2,3,Paso Robles,November 15,"$489,900",1072934,2263,240 Cheyenne
9,7,9,Paso Robles,November 15,"$3,500,000",1072905,5555,1565 Kiler Canyon
